In [1]:
import pandas as pd

the_movies = pd.read_pickle('data/tmdb-movies-220915-clean.pkl')
the_movies.head()

,title,genres,original_language,popularity,release_date,vote_count,keywords,poster_path,Thriller,Horror,...,Comedy,TV Movie,Crime,Mystery,War,Romance,Music,History,Documentary,Western
id,,,,,,,,,,,,,,,,,,,,,
760741,Beast,"[Thriller, Horror]",en,5056.663,2022-08-11,284,['wild animal'],/iRV0IB5xQeOymuGGUBarTecQVAl.jpg,True,True,...,False,False,False,False,False,False,False,False,False,False
610150,Dragon Ball Super: Super Hero,"[Animation, Science Fiction, Action]",ja,3762.862,2022-06-11,1563,"['android', 'sequel', 'attack', 'based on mang...",/rugyJdeoJm7cSJL1q4jBpTNbxyU.jpg,False,False,...,False,False,False,False,False,False,False,False,False,False
579974,RRR,"[Action, Drama]",te,3684.786,2022-03-24,331,"['revolution', 'liberation', 'freedom fighter'...",/wE0I6efAW4cDDmZQWtwZMOW44EJ.jpg,False,False,...,False,False,False,False,False,False,False,False,False,False
532639,Pinocchio,"[Fantasy, Adventure, Family]",en,3680.307,2022-09-07,411,"['fairy', 'musical', 'puppet', 'live action an...",/g8sclIV4gj1TZqUpnL82hKOTK3B.jpg,False,False,...,False,False,False,False,False,False,False,False,False,False
616037,Thor: Love and Thunder,"[Fantasy, Action, Comedy]",en,3643.289,2022-07-06,3233,"['ex', 'girlfriend', 'hero', 'greek mythology'...",/pIkRyD18kl4FhoCNQuWxWu5cBLM.jpg,False,False,...,True,False,False,False,False,False,False,False,False,False


In [2]:
url_prefix = 'https://image.tmdb.org/t/p/w185'

for url in the_movies['poster_path'][0:5]:
    print(url_prefix + url)

https://image.tmdb.org/t/p/w185/iRV0IB5xQeOymuGGUBarTecQVAl.jpg
https://image.tmdb.org/t/p/w185/rugyJdeoJm7cSJL1q4jBpTNbxyU.jpg
https://image.tmdb.org/t/p/w185/wE0I6efAW4cDDmZQWtwZMOW44EJ.jpg
https://image.tmdb.org/t/p/w185/g8sclIV4gj1TZqUpnL82hKOTK3B.jpg
https://image.tmdb.org/t/p/w185/pIkRyD18kl4FhoCNQuWxWu5cBLM.jpg


In [3]:
the_movies['genres'].value_counts()

[Documentary]                                            56800
[Drama]                                                  53411
[Comedy]                                                 30953
[Animation]                                              13071
[Music]                                                  11897
                                                         ...  
[Mystery, Thriller, TV Movie, Drama]                         1
[Science Fiction, Adventure, Action, Drama, TV Movie]        1
[Mystery, Comedy, Drama, Fantasy]                            1
[TV Movie, Horror, Science Fiction, Thriller]                1
[Animation, Crime, Horror]                                   1
Name: genres, Length: 10072, dtype: int64

In [4]:
import requests
from tqdm import tqdm

urls=[]
for i in tqdm(range(the_movies.shape[0])):
    url_path = url_prefix + the_movies['poster_path'][i:i+1].values[0]
    file_name = './data/tmdb/' + str(the_movies['poster_path'][i:i+1].index[0]) + '.jpg'
    urls.append((file_name, url_path))

100%|██████████| 348035/348035 [00:24<00:00, 14415.63it/s]


In [20]:
def download_poster(url):
    file_name, url_path = url
    r = requests.get(url_path, stream=True)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        print(url_path)


In [ ]:
from multiprocessing.pool import ThreadPool
ThreadPool(9).imap_unordered(download_poster, urls)

In [6]:
from os import listdir
from os.path import isfile, join
downloaded_files = ['./data/tmdb/' + f for f in listdir('./data/tmdb/') if isfile(join('./data/tmdb/', f))]

In [8]:
images = list(list(zip(*urls))[0])

In [13]:
missing = set(images) - set(downloaded_files)

In [18]:
urls_missing = []
for i in tqdm(range(len(urls))):
    file_name = urls[i][0]
    url_path = urls[i][1]
    if urls[i][0] in missing:
        urls_missing.append((file_name, url_path))

100%|██████████| 348035/348035 [00:01<00:00, 304970.91it/s]


In [21]:
for url in urls_missing:
    download_poster(url)